# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
D. Christie  ->  D. Christie  |  ['D. Christie']
L. Acuna  ->  L. Acuna  |  ['L. Acuna']
J. Li  ->  J. Li  |  ['J. Li']
S. Li  ->  S. Li  |  ['S. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Arxiv has 64 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2403.03257


extracting tarball to tmp_2403.03257...

 done.


H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


...

extracting tarball to tmp_2403.03845...

 done.
Retrieving document from  https://arxiv.org/e-print/2403.03872


extracting tarball to tmp_2403.03872...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2403.03872/tmp.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'journal_abbrev' from 'tmp_2403.03872/journal_abbrev.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.03325-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.03325) | **JWST Reveals CH$_4$, CO$_2$, and H$_2$O in a Metal-rich Miscible  Atmosphere on a Two-Earth-Radius Exoplanet**  |
|| B. Benneke, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>D. Christie</mark>, <mark>L. Acuna</mark> |
|*Appeared on*| *2024-03-07*|
|*Comments*| *25 pages, 12 figures*|
|**Abstract**| Even though sub-Neptunes likely represent the most common outcome of planet formation, their natures remain poorly understood. In particular, planets near 1.5-2.5$\,R_\oplus$ often have bulk densities that can be explained equally well with widely different compositions and interior structures, resulting in grossly divergent implications for their formation. Here, we present the full 0.6-5.2$\,\mu \mathrm{m}$ JWST NIRISS/SOSS+NIRSpec/G395H transmission spectrum of the 2.2$\,R_\oplus$ TOI-270d ($4.78\,M_\oplus$, $T_\mathrm{eq}$=350-380 K), delivering unprecedented sensitivity for atmospheric characterization in the sub-Neptune regime. We detect five vibrational bands of CH$_4$ at 1.15, 1.4, 1.7, 2.3, and 3.3$\,\mu$m (9.4$\sigma$), the signature of CO$_2$ at 4.3$\,\mu$m (4.8$\sigma$), water vapor (2.5$\sigma$), and potential signatures of SO$_2$ at 4.0$\,\mu \mathrm{m}$ and CS$_2$ at 4.6$\,\mu\mathrm{m}$. Intriguingly, we find an overall highly metal-rich atmosphere, with a mean molecular weight of $5.47_{-1.14}^{+1.25}$. We infer an atmospheric metal mass fraction of $58_{-12}^{+8}\%$ and a C/O of $0.47_{-0.19}^{+0.16}$, indicating that approximately half the mass of the outer envelope is in high-molecular-weight volatiles (H$_2$O, CH$_4$, CO, CO$_2$) rather than H$_2$/He. We introduce a sub-Neptune classification scheme and identify TOI-270d as a "miscible-envelope sub-Neptune" in which H$_2$/He is well-mixed with the high-molecular-weight volatiles in a miscible supercritical metal-rich envelope. For a fully miscible envelope, we conclude that TOI-270d's interior is $90_{-4}^{+3}\,$wt$\,\%$ rock/iron, indicating that it formed as a rocky planet that accreted a few wt % of H$_2$/He, with the overall envelope metal content explained by magma-ocean/envelope reactions without the need for significant ice accretion. TOI-270d may well be an archetype of the overall population of sub-Neptunes. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.03379-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.03379) | **The ALMA-CRISTAL survey: Extended [CII] emission in an interacting  galaxy system at z ~ 5.5**  |
|| A. Posses, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-03-07*|
|*Comments*| *Submitted to A&A - comments are welcome! - 19 pages, 13 figures*|
|**Abstract**| The ALMA [CII] Resolved Ism in STar-forming gALaxies (CRISTAL) survey is a Cycle 8 ALMA Large Programme that studies the cold gas component of high-redshift galaxies. Its sub-arcsecond resolution observations are key to disentangling physical mechanisms that shape galaxies during cosmic dawn. In this paper, we explore the morphology and kinematics of the cold gas, star-forming, and stellar components in the star-forming main-sequence galaxy CRISTAL-05/HZ3, at z = 5.54. Our analysis includes 0.3" spatial resolution (~2 kpc) ALMA observations of the [CII] line. While CRISTAL-05 was previously classified as a single source, our observations reveal that the system is a close interacting pair surrounded by an extended component of carbon-enriched gas. This is imprinted in the disturbed elongated [CII] morphology and the separation of the two components in the position-velocity diagram (~100 km/s). The central region is composed of two components, named C05-NW and C05-SE, with the former being the dominant one. A significant fraction of the [CII] arises beyond the close pair up to 10 kpc, while the regions forming new massive stars and the stellar component seem compact (r_[CII] ~ 4 r_UV), as traced by rest-frame UV and optical imaging obtained with the Hubble Space Telescope and the James Webb Space Telescope. Our kinematic model, using the DYSMALpy software, yields a minor contribution of dark matter of C05-NW within a radius of ~2x Reff. Finally, we explore the resolved [CII]/FIR ratios as a proxy for shock-heating produced by this merger. We argue that the extended [CII] emission is mainly caused by the merger, which could not be discerned with lower-resolution observations. Our work emphasizes the need for high-resolution observations to fully characterize the dynamic stages of infant galaxies and the physical mechanisms that drive the metal enrichment of the circumgalactic medium. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.03845-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.03845) | **Surveys of clumps, cores, and condensations in Cygnus X: Temperature and  nonthermal velocity dispersion revealed by VLA NH3 observations**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2024-03-07*|
|*Comments*| *37 pages, 19 figures*|
|**Abstract**| The physical properties, evolution, and fragmentation of massive dense cores (MDCs, $\sim$ 0.1 pc) are fundamental pieces in our understanding of high-mass star formation. We aim to characterize the temperature, velocity dispersion, and fragmentation of the MDCs in the Cygnus X giant molecular cloud and to investigate the stability and dynamics of these cores. We present the Karl G. Jansky Very Large Array (VLA) observations of the NH$_3$ (J,K) = (1,1) and (2,2) inversion lines towards 35 MDCs in Cygnus X, from which we calculated the temperature and velocity dispersion. We extracted 202 fragments ($\sim$ 0.02 pc) from the NH$_3$ (1,1) moment-0 maps with the GAUSSCLUMPS algorithm. We analyzed the stability of the MDCs and their NH$_3$ fragments through evaluating the corresponding kinetic, gravitational potential, and magnetic energies and the virial parameters. The MDCs in Cygnus X have a typical mean kinetic temperature T$_K$ of $\sim$ 20 K. Our virial analysis shows that many MDCs are in subvirialized states, indicating that the kinetic energy is insufficient to support these MDCs against their gravity. The calculated nonthermal velocity dispersions of most MDCs are at transonic to mildly supersonic levels, and the bulk motions make only a minor contribution to the velocity dispersion. Regarding the NH$_3$ fragments, with T$_K$ $\sim$ 19 K, their nonthermal velocity dispersions are mostly trans-sonic to subsonic. Unless there is a strong magnetic field, most NH$_3$ fragments are probably not in virialized states. We also find that most of the NH$_3$ fragments are dynamically quiescent, while only a few are active due to star formation activity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.03872-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.03872) | **A dormant, overmassive black hole in the early Universe**  |
|| I. Juodžbalis, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-03-07*|
|*Comments*| *38 pages, 18 figures. Submitted*|
|**Abstract**| Recent observations have found a large number of supermassive black holes already in place in the first few hundred million years after Big Bang. The channels of formation and growth of these early, massive black holes are not clear, with scenarios ranging from heavy seeds to light seeds experiencing bursts of high accretion rate. Here we present the detection, from the JADES survey, of broad Halpha emission in a galaxy at z=6.68, which traces a black hole with mass of ~ 4 * 10^8 Msun and accreting at a rate of only 0.02 times the Eddington limit. The host galaxy has low star formation rate (~ 1 Msun/yr, a factor of 3 below the star forming main sequence). The black hole to stellar mass ratio is ~ 0.4, i.e. about 1,000 times above the local relation, while the system is closer to the local relations in terms of dynamical mass and velocity dispersion of the host galaxy. This object is most likely the tip of the iceberg of a much larger population of dormant black holes around the epoch of reionisation. Its properties are consistent with scenarios in which short bursts of super-Eddington accretion have resulted in black hole overgrowth and massive gas expulsion from the accretion disk; in between bursts, black holes spend most of their life in a dormant state. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.03257-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.03257) | **Kinematical Fluctuations Vary with Galaxy Surface Mass Density**  |
|| Z.-H. Zhong, G. Zhao, <mark>H.-W. Rix</mark>, L. C. Ho |
|*Appeared on*| *2024-03-07*|
|*Comments*| *13 pages,10 figures, accepted for publication in ApJL*|
|**Abstract**| The Galaxy inner parts are generally considered to be optically symmetric, as well as kinematically symmetric for most massive early-type galaxies. At the lower-mass end, many galaxies contain lots of small patches in their velocity maps, causing their kinematics to be nonsmooth in small scales and far from symmetry. These small patches can easily be mistaken for measurement uncertainties and have not been well discussed. We used the comparison of observations and numerical simulations to demonstrate the small patches existence beyond uncertainties. For the first time we have found that the fluctuation degrees have an approximate inverse loglinear relation with the galaxy stellar surface mass densities. This tight relation among galaxies that do not show obvious optical asymmetry that traces environmental perturbations indicates that stellar motion in galaxies has inherent asymmetry besides external environment influences. The degree of the kinetic asymmetry is closely related to and constrained by the intrinsic properties of the host galaxy. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.03437-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.03437) | **Dark Dragon Breaks Magnetic Chain: Dynamical Substructures of IRDC  G28.34 Form in Supported Environments**  |
|| J. Liu, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-03-07*|
|*Comments*| *35 pages, 24 figures. Accepted by ApJ for publication*|
|**Abstract**| We have comprehensively studied the multi-scale physical properties of the infrared dark cloud (IRDC) G28.34 (the Dragon cloud) with dust polarization and molecular line data from Planck, FCRAO-14m, JCMT, and ALMA. We find that the averaged magnetic fields of clumps tend to be either parallel with or perpendicular to the cloud-scale magnetic fields, while the cores in clump MM4 tend to have magnetic fields aligned with the clump fields. Implementing the relative orientation analysis (for magnetic fields, column density gradients, and local gravity), Velocity Gradient Technique, and modified Davis-Chandrasekhar-Fermi analysis, we find that: G28.34 is located in a trans-to-sub-Alfv\'{e}nic environment ($\mathcal{M}_{A}=0.74$ within $r=15$ pc); the magnetic field is effectively resisting gravitational collapse in large-scale diffuse gas, but is distorted by gravity within the cloud and affected by star formation activities in high-density regions; and the normalized mass-to-flux ratio tends to increase with density and decrease with radius. Considering both the magnetic and turbulent support, we find that the environmental gas of G28.34 is in a super-virial (supported) state, the infrared dark clumps may be in a near-equilibrium state, and core MM4-core4 is in a sub-virial (gravity-dominant) state. In summary, we suggest that magnetic fields dominate gravity and turbulence in the cloud environment at large scales, resulting in relatively slow cloud formation and evolution processes. Within the cloud, gravity could overwhelm both magnetic fields and turbulence, allowing local dynamical star formation to happen. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2403.03325.md
    + _build/html/tmp_2403.03325/figures/TransmissionSpectrum.png
    + _build/html/tmp_2403.03325/figures/Temp_vs_InteriorStructure.png
    + _build/html/tmp_2403.03325/figures/gcm_pt_profiles.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\jqsrt}{JQSRT}$
$\newcommand{\aj}{AJ}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjl}{ApJL}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\aap}{A\&Aj}$
$\newcommand{\araa}{ARAA}$
$\newcommand{\nat}{Nature}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\vect}[1]{\mathbf{#1}}$
$\newcommand{\mt}[1]{\mathrm{#1}}$
$\newcommand{\Msun}{\ensuremath{ M_{\odot}}}$
$\newcommand{\Mearth}{\ensuremath{ M_{\oplus}}}$
$\newcommand{\Mmoon}{\ensuremath{ M_{\mathrm{Moon}}}}$
$\newcommand{\Mjup}{\ensuremath{ M_{\mathrm{Jup}}}}$
$\newcommand{\Rjup}{\ensuremath{ R_{\mathrm{Jup}}}}$
$\newcommand{\mum}{\ensuremath{\mathrm{ \mu m}}}$
$\newcommand{\AMMF}{\ensuremath{\mathrm{AMMF}}}$
$\newcommand{\Zatm}{\ensuremath{Z_{\mathrm{atm}}}}$
$\newcommand{\Zenv}{\ensuremath{Z_{\mathrm{env}}}}$
$\newcommand{\farcmin}{\mbox{\ensuremath{.\mkern-4mu^\prime}}}$
$\newcommand{\farcsec}{\mbox{\ensuremath{ .\!\!^{\prime\prime}}}}$
$\newcommand{\water}{H_{2}O}$
$\newcommand{\logX}[1]{\ensuremath{\log(\mathrm{X_{\ce{#1}}})}}$
$\newcommand{\logXratio}[2]{\ensuremath{\log(\mathrm{X_{\ce{#1}} / X_{\ce{#2}} })}}$
$\newcommand{\rstar}{R_*}$
$\newcommand{\re}{ R_\oplus}$
$\newcommand{\me}{ M_\oplus}$
$\newcommand{\rsun}{ R_\odot}$
$\newcommand{\project}[1]{\textsl{#1}}$
$\newcommand{\JWST}{\project{JWST}}$
$\newcommand{\HST}{\project{HST}}$
$\newcommand{\Hubble}{\project{Hubble}}$
$\newcommand{\Spitzer}{\project{Spitzer}}$
$\newcommand{\Kepler}{\project{Kepler}}$
$\newcommand{\Ktwo}{\project{K2}}$
$\newcommand{\TESS}{\project{TESS}}$
$\newcommand{\CHEOPS}{\project{CHEOPS}}$
$\newcommand$
$\newcommand$
$\newcommand{\umontreal}{Department of Physics and Trottier Institute for Research on Exoplanets, Université de Montréal, Montreal, QC, Canada \href{mailto:bjorn.benneke@umontreal.ca}{bjorn.benneke@umontreal.ca}}$</div>



<div id="title">

# $\large{_JWST_ Reveals $CH_4$, $CO_2$, and $H_2$O in a Metal-rich Miscible Atmosphere\ on a Two-Earth-Radius Exoplanet}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.03325-b31b1b.svg)](https://arxiv.org/abs/2403.03325)<mark>Appeared on: 2024-03-07</mark> -  _25 pages, 12 figures_

</div>
<div id="authors">

B. Benneke, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>D. Christie</mark>, <mark>L. Acuna</mark>

</div>
<div id="abstract">

**Abstract:** Even though sub-Neptunes likely represent the most common outcome of planet formation, their natures remain poorly understood. In particular, planets near 1.5--2.5 $\re$ often have bulk densities that can be explained equally well with widely different compositions and interior structures, resulting in grossly divergent implications for their formation and potential habitability. Here, we present the full 0.6--5.2 $ \mu \mathrm{m}$ _JWST NIRISS/SOSS_ + _NIRSpec/G395H_ transmission spectrum of the 2.2 $ \re$ planet TOI-270 d ( $4.78 \me$ , $T_\mathrm{eq}$ =350--380 K), delivering unprecedented sensitivity for atmospheric characterization in the sub-Neptune regime. We detect five vibrational bands of $CH_4$ at 1.15, 1.4, 1.7, 2.3, and 3.3 $\mu$ m (9.4 $\sigma$ ), the signature of $CO_2$ at 4.3 $\mu$ m (4.8 $\sigma$ ), water vapor (2.5 $\sigma$ ), and potential signatures of $SO_2$ at 4.0 $ \mu \mathrm{m}$ and $CS_2$ at 4.6 $ \mu\mathrm{m}$ . Intriguingly, we find an overall highly metal-rich atmosphere, with a mean molecular weight of $5.47_{-1.14}^{+1.25}$ . We infer an atmospheric metal mass fraction of $58_{-12}^{+8}\%$ and a C/O of $0.47_{-0.19}^{+0.16}$ , indicating that approximately half the mass of the outer envelope is in high-molecular-weight volatiles ($H_2$ O, $CH_4$ , CO, $CO_2$ ) rather than $H_2$ /He. We introduce a sub-Neptune classification scheme and identify TOI-270 d as a "miscible-envelope sub-Neptune" in which $H_2$ /He is well-mixed with the high-molecular-weight volatiles in a miscible supercritical metal-rich envelope. For a fully miscible envelope, we conclude that TOI-270 d's interior is $90_{-4}^{+3}$ wt \% rock/iron, indicating that it formed as a rocky planet that accreted a few wt \% of $H_2$ /He, with the overall envelope metal content explained by magma-ocean/envelope reactions without the need for significant ice accretion. TOI-270 d may well be an archetype of the overall population of sub-Neptunes.

</div>

<div id="div_fig1">

<img src="tmp_2403.03325/figures/TransmissionSpectrum.png" alt="Fig8" width="100%"/>

**Figure 8. -** _JWST/NIRISS + NIRSpec_ transmission spectrum of the temperate $2.2 R_\oplus$ exoplanet TOI-270 d. The top panel shows the observed transit depths (black points) compared to a random sampling of the model transmission spectra in the atmospheric retrieval posterior (blue), with the overall best-fitting model shown in red. The middle panel illustrates the contributions of individual molecular absorbers in the atmosphere of TOI-270 d, with a zoom on the NIRSpec G395H data shown in the bottom panel. Strong vibrational bands of methane gas are detected at 1.15, 1.4, 1.7, 2.3, and 3.3 $\mu$m (purple), with $H_2$O absorption contributing as well at 1.4 and 1.9$ \mu$m. In addition, $CO_2$ absorption is detected at 4.2--4.5 $\mu$m (green) and the increased transit depth around 4.7 $\mu$m is a possible signature of $CS_2$(cyan). The inclusion of $SO_2$ improves the fit at 4.1 $\mu$m. The combined effect of cloud and haze opacity, which becomes significant at wavelengths below 1.1 $\mu$m, is illustrated through the lower gray envelope of the model spectra. (*fig:TransmissionSpectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2403.03325/figures/Temp_vs_InteriorStructure.png" alt="Fig11" width="100%"/>

**Figure 11. -** Temperature-dependent interior structure of sub-Neptunes driven by the phase changes of $H_2$O. TOI-270 d's high atmospheric metal mass fraction indicates that high-molecule-weight volatiles ($H_2$O, $CH_4$, CO, $CO_2$) are well-mixed with the $H_2$/He in a warm miscible envelope (right scenario). The C/H and O/H of the atmosphere is therefore much more representative of the overall envelope composition than for stratified mini-Neptune and hycean worlds. (*fig:Temp_vs_InteriorStructure*)

</div>
<div id="div_fig3">

<img src="tmp_2403.03325/figures/gcm_pt_profiles.png" alt="Fig4" width="100%"/>

**Figure 4. -** Pressure-temperature profiles from the dual-grey GCM model of TOI-270 d, averaged over the final 100 days of the simulation. Profiles at the equator (solid lines) and at a latitude of $\theta=45^{\circ}$ are shown. TOI-270 d is sufficiently warm around the planet that water remains miscible and transitions directly from the vapor form at low pressures into the supercritical phase at high pressure. (*fig:gcm_pt*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.03325"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

379  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
